In [2]:
import torch
from transformers import SeamlessM4TProcessor, SeamlessM4TForTextToText

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
version = "facebook/hf-seamless-m4t-medium"
src_lang = "eng"
tgt_lang = "cmn"
example_english_phrase = "UN Chief Says There Is No Military Solution in Syria"
expected_translation_cmn = "联合国秘书长表示叙利亚问题没有军事解决方案"

# SeamlessM4TProcessor

In [5]:
processor = SeamlessM4TProcessor.from_pretrained(version, src_lang=src_lang)
processor

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


SeamlessM4TProcessor:
- feature_extractor: SeamlessM4TFeatureExtractor {
  "feature_extractor_type": "SeamlessM4TFeatureExtractor",
  "feature_size": 80,
  "language_code": [
    "__ace__",
    "__ace_Latn__",
    "__acm__",
    "__acq__",
    "__aeb__",
    "__afr__",
    "__ajp__",
    "__aka__",
    "__amh__",
    "__apc__",
    "__arb__",
    "__ars__",
    "__ary__",
    "__arz__",
    "__asm__",
    "__ast__",
    "__awa__",
    "__ayr__",
    "__azb__",
    "__azj__",
    "__bak__",
    "__bam__",
    "__ban__",
    "__bel__",
    "__bem__",
    "__ben__",
    "__bho__",
    "__bjn__",
    "__bjn_Latn__",
    "__bod__",
    "__bos__",
    "__bug__",
    "__bul__",
    "__cat__",
    "__ceb__",
    "__ces__",
    "__cjk__",
    "__ckb__",
    "__crh__",
    "__cym__",
    "__dan__",
    "__deu__",
    "__dik__",
    "__dyu__",
    "__dzo__",
    "__ell__",
    "__eng__",
    "__epo__",
    "__est__",
    "__eus__",
    "__ewe__",
    "__fao__",
    "__pes__",
    "__fij__",
    "

## tokenizer([sequence])

In [6]:
inputs = processor(
    text=example_english_phrase,
    padding=True,  # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    return_attention_mask=True,  # 返回attention_mask
    return_tensors="pt",
).to(device)

print(inputs.keys())
print(inputs["input_ids"])
print(inputs["attention_mask"])

dict_keys(['input_ids', 'attention_mask'])
tensor([[256047,  16297, 134408,   8165, 248066,  14734,    950,   1135, 105721,
           3573,     83,  27352,    108,  49486,      3,      0]],
       device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]], device='cuda:0')


# SeamlessM4TForTextToText

The text-to-text SeamlessM4T Model transformer which can be used for T2TT.

In [7]:
model: SeamlessM4TForTextToText = SeamlessM4TForTextToText.from_pretrained(
    version, torch_dtype=torch.float16
).to(device)
model.eval()

SeamlessM4TForTextToText(
  (shared): Embedding(256206, 1024, padding_idx=0)
  (text_encoder): SeamlessM4TEncoder(
    (embed_tokens): Embedding(256206, 1024, padding_idx=0)
    (embed_positions): SeamlessM4TSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-11): 12 x SeamlessM4TEncoderLayer(
        (self_attn): SeamlessM4TAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ffn): SeamlessM4TFeedForwardNetwork(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [8]:
with torch.inference_mode():
    # 指定语言
    outputs = model.generate(**inputs, tgt_lang=tgt_lang)
outputs

tensor([[     3, 256200, 248059, 250844, 251583, 250102,  24042, 249057, 253641,
          87933,  23938, 231527, 120580,  61805,      3]], device='cuda:0')

In [9]:
processor.batch_decode(outputs, skip_special_tokens=True)

['首席说联合国在叙利亚没有军事解决方案']

In [10]:
expected_translation_cmn

'联合国秘书长表示叙利亚问题没有军事解决方案'